In [ ]:
##################################
#Forward Greedy Feature selection#
##################################

import numpy as np
import pandas as pd
import datetime
import time
import os,sys
import gc
from sklearn import *
import lightgbm
from itertools import combinations
import math
from scipy.special import erfinv

DataBaseDir = '../../data'
InputDir = '%s/l0/kfold' % DataBaseDir
MetaInputDir = '%s/meta/kfold' % DataBaseDir
kfold = 5
seed_num = 1
attention_list = ['knn', 'date', 'inter', 'count', 'rolling', 'agg']
verbose = True
has_snapshot = False
datestr = '%s' % datetime.datetime.now().strftime("%Y-%m-%d")

start_time = datetime.datetime.now()
all_feats = []
for attention in attention_list:
    with open('%s/gfs/%s/good_features_2018-01-26.txt' % (DataBaseDir, attention), 'r') as i_file:
        for line in i_file:
            all_feats.append(line.rstrip())
    i_file.close()
print(len(all_feats))

#### loading stage
valid_dfs = []
holdout_dfs = []
test_dfs = []
meta_feats = ['nn_ef', 'knn_2', 'knn_4', 'knn_8', 'knn_16', 'knn_32', 'knn_64', 'knn_128', 'knn_256', 'knn_512', 'knn_1024']
for fold in range(kfold):
    # load raw features
    FoldInputDir = '%s/%s' % (InputDir, fold)
    valid = pd.read_csv('%s/valid.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    # load meta features
    for t in meta_feats:
        FoldOutputDir = '%s/%s' % (MetaInputDir, fold)
        valid_cb_ef = pd.read_csv('%s/valid_%s.csv' % (FoldOutputDir, t), parse_dates= ['visit_date']).reset_index(drop= True)
        valid = pd.concat([valid, valid_cb_ef[[t]]], axis= 1)
    valid['fold'] = fold
    valid_dfs.append(valid)
    print('fold %s done.' % fold)
TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True)
print('==================================')
print('loading data done.')
print('==================================\n')

#### evaluation stage
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

def EvaluateFeature(feats):
    cv_rmlse = .0
    for fold in range(kfold):
        FoldData = {
            'train': TrainData[TrainData['fold'] != fold],
            'valid': TrainData[TrainData['fold'] == fold]
        }
        model = linear_model.ElasticNet(alpha= 0.0004, l1_ratio= 0.2, max_iter= 200, tol= 1e-4, selection= 'random', random_state= 2017)
        model.fit(FoldData['train'][feats].astype(np.float32, copy=False), FoldData['train']['visitors'].values.astype(np.float32, copy=False))
        # for valid
        pred = model.predict(FoldData['valid'][feats])
        rmsle_valid = RMSLE(FoldData['valid']['visitors'].values, pred)
        cv_rmlse += rmsle_valid
    return cv_rmlse/kfold

#all_feats = [c for c in TrainData.columns if(c.startswith(attention))]
all_cols = all_feats.copy()
all_cols.extend(['fold', 'visitors'])
TrainData = TrainData[all_cols]
print('====================================')
print('total features size %s, sample size %s' % (len(all_feats), len(TrainData)))
print('==================================\n')

start = time.time()
score_history = []
good_features = set([])
OutputDir = '%s/gfs/%s' % (DataBaseDir, "all")
if(os.path.exists(OutputDir) == False):
    os.makedirs(OutputDir)
if(has_snapshot):
    with open('%s/good_features_%s.txt' % (OutputDir, datestr), 'r') as o_feat, open('%s/score_history_%s.txt' % (OutputDir, datestr), 'r') as o_score:
        for line in o_feat:
            good_features.add(line.rstrip())
        for line in o_score:
            parts = line.rstrip().split(',')
            score_history.append((float(parts[0]), parts[1]))
    o_feat.close()
    o_score.close()
    print('loading good feature snapshot done.')
while ((len(score_history) < 2) or (score_history[-1][0] < score_history[-2][0])):
    scores = []
    for feature in all_feats:
        if feature not in good_features:
            selected_features = list(good_features) + [feature]
            score = EvaluateFeature(selected_features)
            scores.append((score, feature))
    if(len(scores) == 0):
        break
    selected = sorted(scores)[0]
    current_feat = selected[1]
    current_score = selected[0]
    good_features.add(current_feat)
    score_history.append(selected)
    end = time.time()
    if verbose:
        improved_score = .0
        if(len(score_history) > 1):
            improved_score = score_history[-2][0] - score_history[-1][0]
        print('====================================')
        print('Current master %s, improve score %.5f, time elapsed %.2fs' % (current_feat, improved_score, (end - start)))
        print('====================================\n')
    with open('%s/good_features_%s.txt' % (OutputDir, datestr), 'w') as o_feat, open('%s/score_history_%s.txt' % (OutputDir, datestr), 'w') as o_score:
        for feat in good_features:
            o_feat.write('%s\n' % feat)
        for score, feat in score_history:
            o_score.write('%s,%s\n' % (str(score), feat))
    o_feat.close()
    o_score.close()

# Remove the last added feature if necessary
if(score_history[-1][0] > score_history[-2][0]):
    good_features.remove(score_history[-1][1])
good_features = sorted(list(good_features))
if verbose:
    print("Selected Features : ", good_features)
    
with open('%s/good_features_%s.txt' % (OutputDir, datestr), 'w') as o_feat, open('%s/score_history_%s.txt' % (OutputDir, datestr), 'w') as o_score:
    for feat in good_features:
        o_feat.write('%s\n' % feat)
    for score, feat in score_history:
        o_score.write('%s,%s\n' % (str(score), feat))
o_feat.close()
o_score.close()

120


/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


fold 0 done.
fold 1 done.
fold 2 done.
fold 3 done.
fold 4 done.
loading data done.

total features size 120, sample size 226853

Current master agg_air_store_id_is_up_corner_dow_visitors_mean, improve score 0.00000, time elapsed 87.82s

Current master knn_8, improve score 0.00582, time elapsed 176.80s

Current master next_is_holiday, improve score 0.00107, time elapsed 277.34s

Current master agg_air_store_id_is_up_corner_holiday_flg_visitors_mean, improve score 0.00105, time elapsed 381.71s



/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Current master last_day_holiday, improve score 0.00054, time elapsed 510.32s

Current master is_weekends, improve score 0.00056, time elapsed 643.90s

Current master agg_air_store_id_is_up_corner_is_weekends_visitors_mean, improve score 0.00063, time elapsed 794.49s

Current master dow, improve score 0.00040, time elapsed 1059.01s

Current master knn_2, improve score 0.00037, time elapsed 1354.52s

Current master day, improve score 0.00036, time elapsed 1816.67s

Current master prev_is_holiday, improve score 0.00017, time elapsed 2318.70s

Current master agg_air_store_id_is_up_corner_prev_is_holiday_visitors_median, improve score 0.00013, time elapsed 2753.12s

Current master agg_air_store_id_is_up_corner_hol_days_visitors_median, improve score 0.00017, time elapsed 3277.92s

Current master rolling_area_genre_visitors_46_39_rolling_area_genre_reserve_visitors_46_39_diff, improve score 0.00012, time elapsed 3851.87s

Current master rolling_id_visitors_46_39, improve score 0.00013, time 